# Part I — Foundations  
## 5. Django Architecture Overview (How Django Really Works End-to-End)

This chapter is the “wiring diagram” of Django. You’ll learn what happens:

- when Django starts,
- when a request comes in,
- how URLs find views,
- how middleware wraps the request/response,
- how templates are located and rendered,
- where settings affect behavior,
- how WSGI/ASGI fit into deployment,
- and how static/media differ (and why it matters in production).

If you understand this chapter deeply, you’ll stop “copy-pasting Django patterns”
and start *designing* Django systems.

---

## 5.0 Learning Outcomes

By the end of this chapter, you should be able to:

1. Explain Django’s core components: project, apps, settings, URLconf, views,
   templates, models, middleware.
2. Trace a request from the web server to Django and back (including middleware).
3. Explain MTV (Model–Template–View) and how it maps to MVC.
4. Explain what `manage.py` does and why it’s used.
5. Explain WSGI vs ASGI (what they are, when you choose which).
6. Explain static vs media files and how they’re served in dev vs production.
7. Set up a simple custom middleware and prove it runs.
8. Understand environments (dev/staging/prod) and why settings must be managed via
   environment variables.

---

## 5.1 Django’s Big Picture: “A Configured Request/Response Engine”

Django is essentially:

- a startup phase where it loads configuration and builds an application object
- a request phase where it:
  - runs middleware
  - resolves URL → view
  - returns response
- a response phase where it:
  - runs middleware (reverse order)
  - returns response to server

### 5.1.1 The core loop in Django terms

```text
HTTP Request
  -> Django middleware (in order)
  -> URL resolver
  -> View (your code)
  -> Template rendering / ORM / business logic (your code + Django)
  -> HttpResponse / JsonResponse / RedirectResponse
  -> Django middleware (reverse order)
HTTP Response
```

That’s the whole system. Everything else is “helpers around this loop.”

---

## 5.2 MTV Explained (and Why It’s Not Actually Confusing)

Django uses the term **MTV**:

- **Model**: data + database mapping (ORM)
- **Template**: presentation (HTML templates)
- **View**: the request handler (the function/class you write)

People coming from “MVC” sometimes get confused because in many MVC frameworks:

- **Controller** handles request logic
- **View** is the HTML template

Django’s naming is basically:

- Django **View** ≈ MVC **Controller**
- Django **Template** ≈ MVC **View**

### 5.2.1 A minimal MTV example (with clear mapping)

**Model** (later chapters, preview only):

```python
# blog/models.py
from django.db import models


class Article(models.Model):
    title = models.CharField(max_length=200)
    body = models.TextField()
```

**View** (request handler):

```python
# blog/views.py
from django.http import HttpResponse


def hello(request):
    return HttpResponse("Hello")
```

**Template** (presentation):

```django
{# blog/templates/blog/article_detail.html #}
<h1>{{ article.title }}</h1>
<div>{{ article.body }}</div>
```

Django “view” decides:
- which template to render
- what context (data) to pass into it

---

## 5.3 Startup: What Happens When Django Boots

When you run:

```bash
python manage.py runserver
```

or:

```bash
python manage.py test
```

Django must:

1. Locate your settings module
2. Load settings
3. Build an app registry from `INSTALLED_APPS`
4. Import app modules as needed (models, admin, etc.)
5. Prepare URL configuration (not always fully resolved immediately, but available)
6. Start serving requests (runserver) or run commands (migrate/test/etc.)

### 5.3.1 The role of `manage.py` (why it exists)

Open your `manage.py`. It typically does two key things:

- sets the environment variable `DJANGO_SETTINGS_MODULE`
- calls Django’s command runner

It’s what makes this work:

```bash
python manage.py migrate
```

Without `manage.py`, Django wouldn’t know which settings to use unless you set
environment variables manually.

### 5.3.2 What is `DJANGO_SETTINGS_MODULE`?

It’s a pointer to the Python module path for settings, like:

```text
config.settings
```

Meaning:
- import `config/settings.py` as your settings source

You can prove it exists at runtime:

```python
# anywhere in Django runtime, e.g., a view
import os

value = os.environ.get("DJANGO_SETTINGS_MODULE")
```

In production, your process manager sets it, or your WSGI/ASGI entry point sets it.

---

## 5.4 Settings: The Central Control Panel

Settings are not “just config.” They affect:

- security behavior (CSRF, cookies, allowed hosts)
- what apps exist
- middleware pipeline
- database connections
- templates
- static/media file behavior

### 5.4.1 Settings are Python code (powerful and dangerous)

Because `settings.py` is Python, you can do:

```python
import os

DEBUG = os.environ.get("DEBUG", "false").lower() == "true"
```

This is flexible, but it also means:
- don’t put heavy logic in settings
- don’t do network calls at import time
- keep settings predictable and environment-driven

### 5.4.2 The settings you must understand early

#### `INSTALLED_APPS`
List of apps Django loads. It controls:
- app registry
- model discovery/migrations
- template discovery (when `APP_DIRS=True`)
- static file discovery (via `django.contrib.staticfiles`)

Example (yours + Django defaults):

```python
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "pages",
]
```

#### `MIDDLEWARE`
Middleware wraps every request/response. Order matters.

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]
```

Key idea: middleware is a pipeline.

- earlier middleware sees request first
- later middleware sees response first (reverse order on the way out)

#### `TEMPLATES`
Controls how templates are found and rendered.

Important keys:
- `DIRS`: explicit template directories
- `APP_DIRS`: whether Django auto-searches templates inside installed apps
- `context_processors`: injects common variables into templates

#### `DATABASES`
Defines database connection settings (SQLite dev default).

#### `STATIC_URL`, `MEDIA_URL`
Defines URL prefixes for serving static and media.

---

## 5.5 Apps: What an “App” Really Is (Beyond “a Folder”)

An app is a feature package that can contain:

- models (data)
- views (request handlers)
- templates (presentation)
- URLconf (routing)
- admin registrations
- tests
- migrations
- services/utilities

### 5.5.1 App config (`apps.py`)
Each app has an `AppConfig` (generated in `pages/apps.py`).

Example (typical):

```python
from django.apps import AppConfig


class PagesConfig(AppConfig):
    default_auto_field = "django.db.models.BigAutoField"
    name = "pages"
```

Django uses this to register the app properly.

### 5.5.2 The app registry
When Django starts, it builds an internal registry of installed apps. This enables:
- model loading
- signal connections (later)
- admin discovery (later)
- checks framework

Practical consequence:
- if you forget to add your app to `INSTALLED_APPS`, it often “exists on disk” but
  doesn’t fully participate in the system.

---

## 5.6 URL Routing: How Django Finds the Correct View

Routing is how Django maps:

```text
GET /healthz/
```

to:

```python
pages.views.healthz
```

### 5.6.1 URLconf layers (project + app)
You normally have:

- `config/urls.py` (project-level)
- `pages/urls.py` (app-level)

Project-level includes app-level:

```python
# config/urls.py
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("pages.urls")),
]
```

App-level maps paths to views:

```python
# pages/urls.py
from django.urls import path

from . import views

urlpatterns = [
    path("", views.home, name="home"),
    path("healthz/", views.healthz, name="healthz"),
]
```

### 5.6.2 Why `include(...)` is an industry standard pattern
It keeps code modular:

- Project-level routing stays readable.
- Each app owns its routes.
- You can move an app’s URLs under a prefix later without rewriting everything.

Example (later refactor):

```python
path("pages/", include("pages.urls"))
```

Now `/healthz/` becomes `/pages/healthz/` automatically.

---

## 5.7 Views: The “Controller” Layer (Where Request Meets Business Logic)

A Django view gets:

- `request` (an `HttpRequest` object)
- and returns `HttpResponse` (or subclass)

### 5.7.1 The `request` object: what’s inside (practical introspection)

Add a debugging view temporarily (for learning):

```python
# pages/views.py
from django.http import JsonResponse


def request_debug(request):
    return JsonResponse(
        {
            "method": request.method,
            "path": request.path,
            "full_path": request.get_full_path(),
            "query_params": dict(request.GET),
            "content_type": request.content_type,
            "user_is_authenticated": getattr(
                request.user, "is_authenticated", False
            ),
        }
    )
```

Wire it:

```python
# pages/urls.py
path("request-debug/", views.request_debug, name="request_debug"),
```

Now try:

```bash
curl -i "http://127.0.0.1:8000/request-debug/?page=2&tag=django"
```

**What to learn from each field:**

- `request.method`: GET/POST/etc.
- `request.path`: `/request-debug/` (no query string)
- `request.get_full_path()`: includes query string
- `request.GET`: parsed query params (strings; can have multiple values)
- `request.content_type`: matters for POST bodies (JSON vs form)
- `request.user`: provided by AuthenticationMiddleware (depends on middleware)

> Note: `dict(request.GET)` gives lists of values because query params can repeat,
> like `?tag=a&tag=b`. You’ll handle this properly later.

---

## 5.8 Middleware: The Wrapper Around Every Request

Middleware is one of Django’s most important “architecture” concepts.

### 5.8.1 What middleware is (concretely)
Middleware is code that runs:

- before the view (on request in)
- after the view (on response out)

Use cases:
- sessions
- authentication
- CSRF protection
- security headers
- logging
- performance measurement
- request IDs
- locale/timezone selection
- tenant resolution (multi-tenancy later)

### 5.8.2 Why order matters (real example)
If SessionMiddleware doesn’t run before AuthenticationMiddleware:
- authentication can’t load the user from the session
- `request.user` may not behave correctly

So middleware order encodes dependencies.

### 5.8.3 Create a timing middleware (hands-on, proves the pipeline)

Create `pages/middleware.py`:

```python
import time

from django.http import HttpRequest, HttpResponse


class RequestTimingMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request: HttpRequest) -> HttpResponse:
        start = time.perf_counter()

        response = self.get_response(request)

        duration_ms = (time.perf_counter() - start) * 1000
        response["X-Request-Duration-Ms"] = f"{duration_ms:.2f}"
        return response
```

#### Explain every part

- `__init__(self, get_response)`:
  - Django constructs middleware once at startup
  - `get_response` is the next layer in the chain
- `__call__(self, request)`:
  - this runs for every request
  - you can do “before view” work before calling `get_response`
  - you can do “after view” work after receiving the response
- We add a response header:
  - `X-Request-Duration-Ms` is a custom header you can inspect with curl/devtools
  - this proves middleware executed and wrapped the view

Register middleware in `config/settings.py` near the top (after SecurityMiddleware is
a reasonable place for this demo):

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "pages.middleware.RequestTimingMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]
```

Test:

```bash
curl -i http://127.0.0.1:8000/healthz/
```

Look for:

```text
X-Request-Duration-Ms: 1.23
```

**What you just proved:**
- request went through middleware
- view ran
- response came back through middleware

This is the same mechanism used by Django’s built-in security, sessions, and CSRF.

---

## 5.9 Templates: How Django Finds and Renders HTML

Even though you haven’t rendered templates yet, you must understand discovery,
because “template not found” errors are common.

### 5.9.1 Template discovery strategies (two main ones)

#### Strategy A: App templates (common in Django apps)
If `APP_DIRS=True` and your app is in `INSTALLED_APPS`, Django searches:

```text
<app_name>/templates/<app_name>/...
```

Example:

```text
pages/
  templates/
    pages/
      home.html
```

This namespacing (`pages/home.html`) prevents collisions between apps.

#### Strategy B: Project-level templates directory
In `TEMPLATES["DIRS"]`, you can add a top-level folder like:

```text
templates/
  base.html
  shared/nav.html
```

This is common for site-wide templates.

### 5.9.2 Preview: Rendering a template vs returning raw HttpResponse
You did:

```python
return HttpResponse("Hello")
```

Later you’ll do:

```python
from django.shortcuts import render

return render(request, "pages/home.html", {"name": "Asha"})
```

What `render(...)` does:
- loads template
- merges context data
- returns an HttpResponse with HTML body

---

## 5.10 Models + ORM (Where Data Lives) — Architecture Preview

You haven’t built models yet, but architecture-wise:

- Models define schema and behavior for data.
- Migrations record schema changes over time.
- ORM provides query language and abstraction over SQL.

The important architectural boundary:
- **Views** orchestrate (handle request, call domain logic, return response)
- **Models/ORM** represent and access data
- **Templates** present data

In professional apps, business logic often sits in:
- services (pure Python functions/modules)
- model methods (carefully, when truly about the model)
- form/serializer validation logic

You’ll learn the best practice patterns later; for now, understand where each piece
fits.

---

## 5.11 WSGI vs ASGI (Deployment Interfaces You Must Understand)

### 5.11.1 What these are (plain language)
WSGI and ASGI are **interfaces** between a Python web app (Django) and a server.

- WSGI: traditional, synchronous request/response (HTTP)
- ASGI: supports async and additional protocols (WebSockets), plus HTTP

Django provides both entry points:

- `config/wsgi.py` for WSGI servers (e.g., Gunicorn in WSGI mode)
- `config/asgi.py` for ASGI servers (e.g., Uvicorn, Daphne)

### 5.11.2 When you choose which (industry guidance)
Choose WSGI if:
- you’re building a classic server-rendered site
- you don’t need WebSockets
- you want simplest stable deployment

Choose ASGI if:
- you need WebSockets/realtime features (often with Django Channels)
- you plan to use async views meaningfully
- your platform standardizes on ASGI

Many teams still deploy with WSGI for classic apps and use ASGI only when needed.

### 5.11.3 What *does not* change
Regardless of WSGI/ASGI, your Django concepts remain:
- URLs, views, templates, models, middleware

ASGI just changes how requests are delivered to Django at the server boundary.

---

## 5.12 Static vs Media Files (And Why Production Handling Is Different)

This topic becomes critical the moment you deploy.

### 5.12.1 Static files: “part of your code”
Static files are assets shipped with your application, such as:
- CSS
- JavaScript
- images used by your UI
- fonts

In Django:
- they are usually collected and served efficiently by Nginx/CDN

Key settings:

```python
STATIC_URL = "static/"
```

Later you’ll also use:
- `STATICFILES_DIRS` (where static files are during development)
- `STATIC_ROOT` (where collected static files are placed for production)

### 5.12.2 Media files: “user-generated content”
Media files are uploaded by users, such as:
- profile photos
- documents
- product images

Key settings later:
- `MEDIA_URL`
- `MEDIA_ROOT`

### 5.12.3 Why static and media are treated differently
Static:
- versioned with your code
- can be cached aggressively (CDN)
- can be rebuilt anytime

Media:
- must be stored persistently
- not rebuilt from code
- needs backup strategy
- often stored in object storage (S3-like) in production

### 5.12.4 Dev vs prod behavior (important)
In development:
- Django can serve static files for convenience
- you might serve media files via Django too

In production:
- Django should not serve static/media directly (performance, security)
- use Nginx/CDN/object storage

---

## 5.13 Environments: Dev, Staging, Production (Settings Strategy)

A real Django system usually has at least:

- **Development**: DEBUG on, local DB, local email backend
- **Staging**: production-like, used to test deploys safely
- **Production**: real users, strict security, monitoring, backups

### 5.13.1 The rule: same code, different config
Your code should not change per environment; configuration should.

That’s why:
- environment variables exist
- secrets must not be hardcoded

### 5.13.2 Minimum environment variables you’ll use later
You’ll eventually manage:
- `DJANGO_SECRET_KEY`
- `DJANGO_DEBUG`
- `DATABASE_URL` or DB host/user/password
- allowed hosts / CSRF trusted origins

We will implement a clean settings strategy in later chapters (deployment/security),
but the architectural principle belongs here.

---

## 5.14 Mini-Lab: Trace the Full Request Flow (Prove You Understand It)

You’ll add a route, a view, middleware, and inspect headers and behavior.

### Step 1: Add a view that uses query params
Add to `pages/views.py`:

```python
from django.http import JsonResponse


def paginate_demo(request):
    page_raw = request.GET.get("page", "1")
    page_size_raw = request.GET.get("page_size", "20")

    try:
        page = int(page_raw)
        page_size = int(page_size_raw)
    except ValueError:
        return JsonResponse(
            {
                "error": "page and page_size must be integers",
                "received": {"page": page_raw, "page_size": page_size_raw},
            },
            status=400,
        )

    if page < 1:
        return JsonResponse({"error": "page must be >= 1"}, status=400)

    if page_size < 1:
        return JsonResponse({"error": "page_size must be >= 1"}, status=400)

    if page_size > 100:
        page_size = 100

    offset = (page - 1) * page_size

    return JsonResponse(
        {
            "page": page,
            "page_size": page_size,
            "offset": offset,
            "note": (
                "This shows why pagination is query-based: "
                "it modifies how the list is returned."
            ),
        }
    )
```

Wire it in `pages/urls.py`:

```python
path("paginate-demo/", views.paginate_demo, name="paginate_demo"),
```

### Step 2: Hit it with curl and inspect headers
Run:

```bash
curl -i "http://127.0.0.1:8000/paginate-demo/?page=2&page_size=50"
```

Observe:
- you got JSON
- you can see `offset`
- you can see your middleware header `X-Request-Duration-Ms` if it’s still enabled

Try invalid input:

```bash
curl -i "http://127.0.0.1:8000/paginate-demo/?page=two"
```

Observe:
- status 400
- error payload explaining what went wrong

**What you just practiced:**
- query parsing
- validation and status codes
- response formatting
- middleware effects
- routing → view

---

## 5.15 Common Architecture Mistakes (And Correct Mental Models)

### Mistake A: “I’ll put everything in views.py”
This works for tiny apps but fails at scale.

Correct approach:
- views orchestrate
- complex logic goes to:
  - services (pure functions/modules)
  - model methods (when truly tied to model)
  - forms/serializers validation (input correctness layer)

### Mistake B: “Templates are just files anywhere”
Template discovery depends on settings (`APP_DIRS`, `DIRS`) and `INSTALLED_APPS`.

Correct approach:
- put app templates inside `app/templates/app/...`
- put shared templates in project-level `templates/` and configure `DIRS`

### Mistake C: “Static and media are the same”
They are not.

Correct approach:
- static: shipped with code, collected, cached
- media: user uploads, persistent storage, backups

### Mistake D: “WSGI vs ASGI changes Django code”
It usually doesn’t change your everyday Django code; it changes deployment boundary.

---

## 5.16 Exercises (Write answers + do small edits)

1. Draw (on paper or in text) the request/response pipeline including:
   - middleware (in + out)
   - URL resolver
   - view

2. Add a new middleware that adds a request ID header:
   - generate a short ID (e.g., from time or random)
   - set `X-Request-Id` in response
   - verify with `curl -i`

3. Explain in your own words:
   - why `INSTALLED_APPS` matters
   - why middleware order matters
   - why templates are namespaced by app

4. Explain why `?page=2&page_size=50` is query-based and list two limitations of
   offset pagination (from Chapter 1).

---

## 5.17 Chapter Summary (The “Permanent” Takeaways)

- Django is a configured request/response engine: middleware → URL routing → view →
  response → middleware.
- `manage.py` + settings load define what Django is and how it behaves.
- Apps are modular feature packages; project wires them together.
- Middleware is a chain; order matters; it’s how sessions/auth/CSRF happen.
- Templates are discovered via settings and app structure.
- WSGI/ASGI are deployment interfaces; they don’t change Django fundamentals.
- Static and media are different categories with different production handling.
- Environments must use the same code with different configuration (env vars).

---

Next: **Part II — Core Django**
- Chapter 6: URL Routing and Views (Function-Based and Class-Based), where we’ll
  go deep into routing patterns, view design, responses, and error handling with
  many real examples.